In [ ]:
import math
import numpy as np
from keras.layers import (
    Dense,
    Activation,
    Dropout,
    Flatten,
    AveragePooling2D,
)
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import Xception
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from google.colab import drive
drive.mount("/content/drive/")

orig_path = '/content/drive/My Drive/bird_classification/' + '/data/bird_dataset/'

Mounted at /content/drive/


In [ ]:
!pip install -U efficientnet

In [ ]:
from efficientnet.keras import EfficientNetB7

In [ ]:
# Seed value (can actually be different for each attribution step)
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value) # tensorflow 2.x
# tf.set_random_seed(seed_value) # tensorflow 1.x

# Original Images

## Loading Data

In [ ]:
img_rows, img_cols = (334,334)
train_batchsize = 16
val_batchsize = 16

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        orig_path + 'train_images/',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bicubic')
 
validation_generator = validation_datagen.flow_from_directory(
        orig_path + 'val_images/',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')

Found 1082 images belonging to 20 classes.
Found 103 images belonging to 20 classes.


## Utils

In [ ]:
# Swish Activation Function
def swish(x):
    return K.sigmoid(x) * x

get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate

# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

## Phase 1 : Trained only on top layer / Xception No Trainable

In [ ]:
base_model = Xception(#EfficientNetB7(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
# Freeze InceptionResNetV2
for layer in base_model.layers:
  layer.trainable = False

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.3)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

In [ ]:
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_xception_freezed.h5",#efficientnet_freezed.h5"
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 1082
nb_validation_samples= 103
epochs=15
batch_size=16

history = model.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
67/67 [==============================] - ETA: 0s - loss: 1.6360 - precision: 0.7009 - recall: 0.3119 - f1: 0.4063 - acc: 0.5206 
Epoch 00001: val_acc improved from -inf to 0.72917, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_freezed.h5
67/67 [==============================] - 939s 14s/step - loss: 1.6360 - precision: 0.7009 - recall: 0.3119 - f1: 0.4063 - acc: 0.5206 - val_loss: 0.8502 - val_precision: 0.7822 - val_recall: 0.6042 - val_f1: 0.6749 - val_acc: 0.7292
Epoch 2/15
67/67 [==============================] - ETA: 0s - loss: 0.9841 - precision: 0.7981 - recall: 0.5761 - f1: 0.6638 - acc: 0.7036
Epoch 00002: val_acc improved from 0.72917 to 0.73958, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_freezed.h5
67/67 [==============================] - 42s 626ms/step - loss: 0.9841 - precision: 0.7981 - recall: 0.5761 - f

In [ ]:
model.load_weights("/content/drive/My Drive/bird_classification/models/inception_xception_freezed.h5")
score = model.evaluate(validation_generator, verbose=1)

7/7 [==============================] - 1s 198ms/step - loss: 0.4390 - precision: 0.8892 - recall: 0.8036 - f1: 0.8394 - acc: 0.8447


- Loss : 0.4390
- Precision : 0.8892
- Recall : 0.8036
- f1_score : 0.8394
- accuracy : 0.8447

## Phase 2 : Retrains last layer of Xception

In [ ]:
base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )
for layer in base_model.layers[:len(base_model.layers)-30]:
  layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-30:]:
  layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_retrain = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_retrain.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

In [ ]:
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_xception_retrained.h5", #efficientnet_retrained.h5
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 1082
nb_validation_samples= 103
epochs=12
batch_size=16

history = model_retrain.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

Epoch 1/12
67/67 [==============================] - ETA: 0s - loss: 1.4078 - precision: 0.7720 - recall: 0.4140 - f1: 0.5198 - acc: 0.5807
Epoch 00001: val_acc improved from -inf to 0.61458, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_retrained.h5
67/67 [==============================] - 44s 656ms/step - loss: 1.4078 - precision: 0.7720 - recall: 0.4140 - f1: 0.5198 - acc: 0.5807 - val_loss: 2.4850 - val_precision: 0.6194 - val_recall: 0.6042 - val_f1: 0.6116 - val_acc: 0.6146
Epoch 2/12
67/67 [==============================] - ETA: 0s - loss: 0.8342 - precision: 0.8433 - recall: 0.6481 - f1: 0.7285 - acc: 0.7448
Epoch 00002: val_acc improved from 0.61458 to 0.69792, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_retrained.h5
67/67 [==============================] - 46s 686ms/step - loss: 0.8342 - precision: 0.8433 - recall: 0.6481 - f1: 0.7285 - acc: 0.7448 - val_loss: 0.9122 - val_precision: 0.7403 - val

In [ ]:
model_retrain.load_weights("/content/drive/My Drive/bird_classification/models/inception_xception_retrained.h5")
model_retrain.evaluate(train_generator, verbose=1)

68/68 [==============================] - 40s 583ms/step - loss: 0.1364 - precision: 0.9734 - recall: 0.9357 - f1: 0.9535 - acc: 0.9575


[0.1363915055990219,
 0.9733805656433105,
 0.935661792755127,
 0.9535019993782043,
 0.9574861526489258]

In [ ]:
model_retrain.evaluate(validation_generator, verbose=1)

7/7 [==============================] - 1s 158ms/step - loss: 0.4043 - precision: 0.9173 - recall: 0.8929 - f1: 0.9044 - acc: 0.9029


[0.404251366853714,
 0.9172619581222534,
 0.8928571343421936,
 0.9043586850166321,
 0.9029126167297363]

- Loss : 0.40425
- Precision : 0.9172
- Recall : 0.8929
- f1_score : 0.9044
- accuracy : 0.9029

Well good news, unfrezing the last layers help us improve the performance of the model. Now let's try to unfreeze 10 less layers so say if it can less overfit.

### Phase 3 : Retrain 10 less layers

In [ ]:
base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )
for layer in base_model.layers[:len(base_model.layers)-20]:
  layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-20:]:
  layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_retrain_v2 = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_retrain_v2.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

In [ ]:
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_xception_retrained_v2.h5", #efficientnet_retrained_v2.h5
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 1082
nb_validation_samples= 103
epochs=15
batch_size=16

history = model_retrain_v2.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

Epoch 1/15
67/67 [==============================] - ETA: 0s - loss: 1.3353 - precision: 0.7297 - recall: 0.4338 - f1: 0.5263 - acc: 0.5863
Epoch 00001: val_acc improved from -inf to 0.75000, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_retrained_v2.h5
67/67 [==============================] - 43s 644ms/step - loss: 1.3353 - precision: 0.7297 - recall: 0.4338 - f1: 0.5263 - acc: 0.5863 - val_loss: 1.0013 - val_precision: 0.7646 - val_recall: 0.7500 - val_f1: 0.7571 - val_acc: 0.7500
Epoch 2/15
67/67 [==============================] - ETA: 0s - loss: 0.7081 - precision: 0.8463 - recall: 0.6951 - f1: 0.7607 - acc: 0.7833
Epoch 00002: val_acc improved from 0.75000 to 0.76042, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_retrained_v2.h5
67/67 [==============================] - 44s 653ms/step - loss: 0.7081 - precision: 0.8463 - recall: 0.6951 - f1: 0.7607 - acc: 0.7833 - val_loss: 0.9895 - val_precision: 0.7703

In [ ]:
model_retrain_v2.load_weights("/content/drive/My Drive/bird_classification/models/inception_xception_retrained_v2.h5")
score = model_retrain_v2.evaluate(validation_generator, verbose=1)

7/7 [==============================] - 1s 157ms/step - loss: 0.3142 - precision: 0.9218 - recall: 0.8929 - f1: 0.9066 - acc: 0.8932


- Loss : 0.3142
- Precision : 0.9218
- Recall : 0.8929
- f1_score : 0.9066
- accuracy : 0.8932

Well the performance are good but not as good as before

# Cropped Images

In [ ]:
img_rows, img_cols = (224,224)
train_batchsize = 16
val_batchsize = 16

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        orig_path + 'train_images_cropped/',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bicubic')
 
validation_generator = validation_datagen.flow_from_directory(
        orig_path + 'val_images_cropped/',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')

Found 941 images belonging to 20 classes.
Found 92 images belonging to 20 classes.


In [ ]:
base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(224,224,3)
    )
for layer in base_model.layers[:len(base_model.layers)-20]:
  layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-20:]:
  layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_retrain_v2 = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_retrain_v2.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_xception_cropped_retrained_v2.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 941
nb_validation_samples= 92
epochs=15
batch_size=16

history = model_retrain_v2.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

Epoch 1/15
58/58 [==============================] - ETA: 0s - loss: 2.2334 - precision: 0.4650 - recall: 0.1685 - f1: 0.2358 - acc: 0.3438
Epoch 00001: val_acc improved from -inf to 0.61250, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_cropped_retrained_v2.h5
58/58 [==============================] - 17s 291ms/step - loss: 2.2334 - precision: 0.4650 - recall: 0.1685 - f1: 0.2358 - acc: 0.3438 - val_loss: 1.7924 - val_precision: 0.6374 - val_recall: 0.6125 - val_f1: 0.6244 - val_acc: 0.6125
Epoch 2/15
58/58 [==============================] - ETA: 0s - loss: 1.2052 - precision: 0.7883 - recall: 0.5025 - f1: 0.6087 - acc: 0.6486
Epoch 00002: val_acc improved from 0.61250 to 0.78750, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_cropped_retrained_v2.h5
58/58 [==============================] - 19s 330ms/step - loss: 1.2052 - precision: 0.7883 - recall: 0.5025 - f1: 0.6087 - acc: 0.6486 - val_loss: 0.7372 - val_p

In [ ]:
score = model_retrain_v2.evaluate(validation_generator, verbose=1)

6/6 [==============================] - 0s 69ms/step - loss: 0.4627 - precision: 0.8590 - recall: 0.8194 - f1: 0.8381 - acc: 0.8478


- Loss : 0.7015
- Precision : 0.8287
- Recall : 0.6458
- f1_score : 0.7226
- accuracy : 0.7500

## Last Layer retrainable

In [ ]:
base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(224,224,3)
    )
for layer in base_model.layers[:len(base_model.layers)-30]:
  layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-30:]:
  layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_retrain = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_retrain.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_xception_cropped_retrained.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 941
nb_validation_samples= 92
epochs=15
batch_size=16

history = model_retrain.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

Epoch 1/15
58/58 [==============================] - ETA: 0s - loss: 2.4251 - precision: 0.5441 - recall: 0.1134 - f1: 0.1774 - acc: 0.2941 
Epoch 00001: val_acc improved from -inf to 0.51250, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_cropped_retrained.h5
58/58 [==============================] - 781s 13s/step - loss: 2.4251 - precision: 0.5441 - recall: 0.1134 - f1: 0.1774 - acc: 0.2941 - val_loss: 2.1108 - val_precision: 0.5463 - val_recall: 0.4750 - val_f1: 0.5076 - val_acc: 0.5125
Epoch 2/15
58/58 [==============================] - ETA: 0s - loss: 1.3537 - precision: 0.7759 - recall: 0.3922 - f1: 0.5116 - acc: 0.5762
Epoch 00002: val_acc improved from 0.51250 to 0.62500, saving model to /content/drive/My Drive/bird_classification/models/inception_xception_cropped_retrained.h5
58/58 [==============================] - 18s 316ms/step - loss: 1.3537 - precision: 0.7759 - recall: 0.3922 - f1: 0.5116 - acc: 0.5762 - val_loss: 1.6467 - val_precisi

In [ ]:
model_retrain.load_weights("/content/drive/My Drive/bird_classification/models/inception_xception_cropped_retrained.h5")
model_retrain.evaluate(train_generator, verbose=1)

59/59 [==============================] - 15s 250ms/step - loss: 0.3344 - precision: 0.9168 - recall: 0.8166 - f1: 0.8621 - acc: 0.8640


[0.33443987369537354,
 0.9168158769607544,
 0.8165742754936218,
 0.86213219165802,
 0.8639745116233826]

In [ ]:

score = model_retrain.evaluate(validation_generator, verbose=1)

6/6 [==============================] - 1s 141ms/step - loss: 0.3988 - precision: 0.9083 - recall: 0.8403 - f1: 0.8682 - acc: 0.8587


- Loss : 0.3988
- Precision : 0.9083
- Recall : 0.8403
- f1_score : 0.8682
- accuracy : 0.8587